In [0]:
# -*- coding: utf-8 -*-
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import pickle
from sklearn.externals import joblib

C:\Users\bong\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:
from sklearn.metrics import accuracy_score

# 데이터 불러오기

In [0]:
data= pd.read_csv("data.csv",encoding = 'utf-8')

In [0]:
data

,Unnamed: 0,열차번호,역,출도착구분,급행여부,지연여부,요일1,환승가능,시간,승차,하차,5번째전역_지연여부,역정체여부,역_시간_실지연가중치
0,0,0,6,0,0,0,1,1,16,1122.0,170.0,0,0,12
1,1,0,7,1,0,0,1,0,16,152.0,43.0,0,0,40
2,2,0,3,1,0,0,1,0,16,711.0,198.0,0,0,1
3,3,0,62,1,0,0,1,0,16,470.0,91.0,0,0,0
4,4,0,65,1,0,0,1,0,16,212.0,58.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808274,808274,1235,85,1,0,1,0,1,5,798.0,825.0,1,1,17
808275,808275,1235,71,1,0,1,0,0,5,1887.0,2223.0,1,1,10
808276,808276,1235,0,1,0,1,0,0,5,417.0,315.0,1,1,5
808277,808277,1235,13,1,0,1,0,0,5,292.0,306.0,1,1,22


In [0]:
data = data.drop(['Unnamed: 0'],axis = 1)

# train, test 데이터 나누기

In [0]:
data_X = data.drop(['지연여부'],axis = 1)

In [0]:
data_Y = data['지연여부']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_Y, test_size=0.2, random_state=1234)

In [0]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646623 entries, 53202 to 486191
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   열차번호         646623 non-null  int64  
 1   역            646623 non-null  int64  
 2   출도착구분        646623 non-null  int64  
 3   급행여부         646623 non-null  int64  
 4   요일1          646623 non-null  int64  
 5   환승가능         646623 non-null  int64  
 6   시간           646623 non-null  int64  
 7   승차           646623 non-null  float64
 8   하차           646623 non-null  float64
 9   5번째전역_지연여부   646623 non-null  int64  
 10  역정체여부        646623 non-null  int64  
 11  역_시간_실지연가중치  646623 non-null  int64  
dtypes: float64(2), int64(10)
memory usage: 64.1 MB


In [0]:
y_train

53202     1
595968    0
755704    1
422665    0
316083    1
         ..
514041    1
220532    1
165158    1
451283    1
486191    1
Name: 지연여부, Length: 646623, dtype: int64

# XGBOOST

In [0]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [0]:
import xgboost as xgb
from xgboost import XGBClassifier

In [0]:
XGBC = XGBClassifier(verbosity = 1 , n_job=-1)
xgc_param_grid={
    'booster': ["gbtree","gblinear"],
    "lambda " : [1, 10,40,100],
    'n_estimators':[10,110,500,1000],
    'learning_rate' : (np.arange(0.1,1,0.3)),
    'colsample_bytree' : (np.arange(0.1,1,0.3))
}

In [0]:
# 랜덤 그리드 서치
n_iter_search = 10
xgb_rgs = RandomizedSearchCV(XGBC,
                            param_distributions = xgc_param_grid,
                            cv=7,
                            n_jobs=-1,
                            scoring='accuracy',
                            verbose=1,
                            random_state = 1234,
                            n_iter = n_iter_search)

xgb_rgs.fit(X_train, y_train)


Fitting 7 folds for each of 10 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 37.4min
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed: 80.7min finished


RandomizedSearchCV(cv=7, error_score=nan,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n...
                                           validate_parameters=False,
                                           verbosity=1),
                   iid='deprecated', n_iter=10, n_job

In [0]:
xgb_rgs.best_params_

{'n_estimators': 1000,
 'learning_rate': 0.4,
 'lambda ': 10,
 'colsample_bytree': 0.4,
 'booster': 'gbtree'}

In [0]:
xgb_rgs.best_score_

0.8406784156278168

In [0]:
xgb_rgs_bt = xgb_rgs.best_estimator_

In [0]:
y_pred = xgb_rgs_bt.predict(X_test)

accuracy_score(y_test,y_pred)

0.8421462859405157

In [0]:
feature_importance = pd.DataFrame({'feature':X_train.columns,
                                  'importance':xgb_rgs_bt.feature_importances_})

feature_importance.sort_values(by='importance',ascending=False)

,feature,importance
9,5번째전역_지연여부,0.654139
2,출도착구분,0.125241
10,역정체여부,0.041793
0,열차번호,0.040450
1,역,0.027938
6,시간,0.022773
3,급행여부,0.020107
11,역_시간_실지연가중치,0.020005
4,요일1,0.016710
8,하차,0.011171


# KNN

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
knn = KNeighborsClassifier()

In [0]:
parameters={
    'n_neighbors':[1,5,10,15],
    'weights':['distance','uniform']    
}

In [0]:
# 랜덤 그리드 서치
n_iter_search = 10
knn_rgs = RandomizedSearchCV(knn,
                            param_distributions = parameters,
                            cv=7,
                            n_jobs=-1,
                            scoring='accuracy',
                            verbose=1,
                            random_state = 1234,
                            n_iter = n_iter_search)

knn_rgs.fit(X_train, y_train)

C:\Users\bong\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 7 folds for each of 8 candidates, totalling 56 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  56 out of  56 | elapsed:  2.5min finished


RandomizedSearchCV(cv=7, error_score=nan,
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'n_neighbors': [1, 5, 10, 15],
                                        'weights': ['distance', 'uniform']},
                   pre_dispatch='2*n_jobs', random_state=1234, refit=True,
                   return_train_score=False, scoring='accuracy', verbose=1)

In [0]:
knn_rgs.best_params_

{'weights': 'uniform', 'n_neighbors': 15}

In [0]:
knn_rgs.best_score_

0.6957191441271545

In [0]:
knn_rgs_bt = knn_rgs.best_estimator_

In [0]:
y_pred = knn_rgs_bt.predict(X_test)

accuracy_score(y_test,y_pred)

0.698470826941159

# DNN

In [0]:
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
import tensorflow.keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# 3.4 텐서플로 2.0 버전 선택
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf


C:\Users\bong\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\bong\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\bong\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\bong\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model3 = tf.keras.Sequential([
    
    #tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Flatten(input_shape=(12,)),
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(2, activation='softmax'),
])


model3.compile(optimizer='Nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
history = model3.fit(X_train, y_train, epochs=50, batch_size=40, validation_split=0.3, callbacks = [cp_callback])

Train on 452636 samples, validate on 193987 samples
Epoch 1/50
452440/452636 [============================>.] - ETA: 0s - loss: 0.6147 - acc: 0.7105
Epoch 00001: saving model to training_1/cp.ckpt
452636/452636 [==============================] - 48s 106us/sample - loss: 0.6146 - acc: 0.7106 - val_loss: 0.4414 - val_acc: 0.7996
Epoch 2/50
452440/452636 [============================>.] - ETA: 0s - loss: 0.4529 - acc: 0.7948
Epoch 00002: saving model to training_1/cp.ckpt
452636/452636 [==============================] - 48s 107us/sample - loss: 0.4529 - acc: 0.7948 - val_loss: 0.4372 - val_acc: 0.7998
Epoch 3/50
452520/452636 [============================>.] - ETA: 0s - loss: 0.4508 - acc: 0.7962
Epoch 00003: saving model to training_1/cp.ckpt
452636/452636 [==============================] - 49s 108us/sample - loss: 0.4508 - acc: 0.7961 - val_loss: 0.4391 - val_acc: 0.8040
Epoch 4/50
452200/452636 [============================>.] - ETA: 0s - loss: 0.4479 - acc: 0.7969- ETA: 1s
Epoch 00004

452200/452636 [============================>.] - ETA: 0s - loss: 0.4394 - acc: 0.8006
Epoch 00030: saving model to training_1/cp.ckpt
452636/452636 [==============================] - 49s 108us/sample - loss: 0.4393 - acc: 0.8006 - val_loss: 0.4386 - val_acc: 0.8025
Epoch 31/50
452440/452636 [============================>.] - ETA: 0s - loss: 0.4384 - acc: 0.8011
Epoch 00031: saving model to training_1/cp.ckpt
452636/452636 [==============================] - 48s 106us/sample - loss: 0.4384 - acc: 0.8011 - val_loss: 0.4375 - val_acc: 0.8012
Epoch 32/50
452480/452636 [============================>.] - ETA: 0s - loss: 0.4433 - acc: 0.7987- ETA: 1s - loss: 0.
Epoch 00032: saving model to training_1/cp.ckpt
452636/452636 [==============================] - 48s 107us/sample - loss: 0.4433 - acc: 0.7987 - val_loss: 0.4347 - val_acc: 0.8012
Epoch 33/50
452240/452636 [============================>.] - ETA: 0s - loss: 0.4384 - acc: 0.8006
Epoch 00033: saving model to training_1/cp.ckpt
452636/45263

In [0]:
model3.evaluate(X_test, y_test, verbose=0)

[0.4459089599828149, 0.8013807]

# lightGBM

In [0]:
X_train.columns = ['trainNumber','station','deparriv','fast','day','re','time','board','unboard','5before','stationdelay','station_time_weight']

In [0]:
X_test.columns = ['trainNumber','station','deparriv','fast','day','re','time','board','unboard','5before','stationdelay','station_time_weight']

In [0]:
X_train

,trainNumber,station,deparriv,fast,day,re,time,board,unboard,5before,stationdelay,station_time_weight
53202,14,6,1,1,0,1,18,397.0,218.0,0,1,50
595968,20,22,3,1,0,0,18,200.0,79.0,0,0,11
755704,12,7,3,1,1,0,17,390.0,127.0,1,0,119
422665,732,43,1,0,0,0,19,725.0,281.0,1,0,130
316083,848,73,1,0,1,0,1,578.0,492.0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
514041,33,14,1,0,1,0,2,248.0,110.0,1,1,4
220532,113,26,1,1,0,0,8,1004.0,1987.0,0,1,1
165158,162,55,1,1,1,2,10,295.0,462.0,1,1,147
451283,875,44,1,0,0,0,10,268.0,403.0,1,1,22


In [0]:
pip install lightgbm

  Using cached lightgbm-2.3.1-py2.py3-none-win_amd64.whl (544 kB)
Note: you may need to restart the kernel to use updated packages.


In [0]:
from lightgbm import LGBMClassifier

In [0]:
LGB = LGBMClassifier(n_jobs=-1)

lb_param_grid = {
    "n_estimators":[100,200,300],
    "learning_rate" :(np.arange(0.1,1,0.3)),
    "max_depth":[-1],
    "min_child_weight":[13],
    "colsample_bytree":(np.arange(0,1,0.3)),
    "num_leaves":(np.arange(6,24,5)),
    "reg_lambda":[1,10],
    "reg_alpha":[1,1.2,0.5],
    "subsample":[0.7,1.0,10]
}

In [0]:
# 랜덤 그리드 서치
n_iter_search = 10
lgb_rgs = RandomizedSearchCV(LGB ,
                            param_distributions = lb_param_grid,
                            cv=7,
                            n_jobs=-1,
                            scoring='accuracy',
                            verbose=1,
                            random_state = 1234,
                            n_iter = n_iter_search)
lgb_rgs.fit(X_train, y_train)

Fitting 7 folds for each of 10 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:  2.4min finished


RandomizedSearchCV(cv=7, error_score=nan,
                   estimator=LGBMClassifier(boosting_type='gbdt',
                                            class_weight=None,
                                            colsample_bytree=1.0,
                                            importance_type='split',
                                            learning_rate=0.1, max_depth=-1,
                                            min_child_samples=20,
                                            min_child_weight=0.001,
                                            min_split_gain=0.0,
                                            n_estimators=100, n_jobs=-1,
                                            num_leaves=31, objective=None,
                                            random_state=None, reg_alpha=0.0,
                                            reg_lambda=0.0, sile...
                   param_distributions={'colsample_bytree': array([0. , 0.3, 0.6, 0.9]),
                                    

In [0]:
lgb_rgs.best_params_

{'subsample': 0.7,
 'reg_lambda': 1,
 'reg_alpha': 1.2,
 'num_leaves': 16,
 'n_estimators': 200,
 'min_child_weight': 13,
 'max_depth': -1,
 'learning_rate': 0.7000000000000001,
 'colsample_bytree': 0.8999999999999999}

In [0]:
lgb_rgs.best_score_

0.829668908696089

In [0]:
lgb_rgs_bt = lgb_rgs.best_estimator_

In [0]:
y_pred = lgb_rgs_bt.predict(X_test)

accuracy_score(y_test,y_pred)

0.8320384025337755

# ensemble

In [0]:
from sklearn.ensemble import VotingClassifier

In [0]:
eclf = VotingClassifier(estimators =
                       [('xgb',xgb_rgs_bt),('lgb',lgb_rgs_bt),('knn',knn_rgs_bt)],
                       voting='soft')

eclf.fit(X_train,y_train)
eclf_pred = eclf.predict(X_test)
accuracy = accuracy_score(y_test,eclf_pred)
print(accuracy)

0.8346983718513387


# Stacking

In [0]:
pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19883 sha256=1892cd9ca6b64ed3b4cf54f4c6a8b24e1fc3f86ea2443f6e17a802aaec0a1e75
  Stored in directory: c:\users\bong\appdata\local\pip\cache\wheels\28\fe\0c\fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack
Note: you may need to restart the kernel to use updated packages.


In [0]:
from vecstack import stacking

In [0]:
# Initialize 1-st level models. 
models = [xgb_rgs_bt, knn_rgs_bt, lgb_rgs_bt] 


# Compute stacking features 
S_train, S_test = stacking(models, X_train, y_train, X_test, 
                           regression = False, metric = accuracy_score, 
                           n_folds = 5, stratified = True, shuffle = True, 
                           random_state = 0, verbose = 2) 


# Initialize 2-nd level model 

model = XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3) 

# Fit 2-nd level model 
model.fit(S_train, y_train) 

# Predict 
y_pred = model.predict(S_test)

accuracy_score(y_pred, y_test)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [XGBClassifier]
    fold  0:  [0.84003866]
    fold  1:  [0.84095109]
    fold  2:  [0.84103615]
    fold  3:  [0.84047818]
    fold  4:  [0.84005289]
    ----
    MEAN:     [0.84051139] + [0.00042509]
    FULL:     [0.84051140]

model  1:     [KNeighborsClassifier]
    fold  0:  [0.69630002]
    fold  1:  [0.69411947]
    fold  2:  [0.69561183]
    fold  3:  [0.69465064]
    fold  4:  [0.69452692]
    ----
    MEAN:     [0.69504178] + [0.00079710]
    FULL:     [0.69504178]

model  2:     [LGBMClassifier]
    fold  0:  [0.82824667]
    fold  1:  [0.83060506]
    fold  2:  [0.82956892]
    fold  3:  [0.83027899]
    fold  4:  [0.83105224]
    ----
    MEAN:     [0.82995038] + [0.00097968]
    FULL:     [0.82995037]



0.8439711486118672